In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# hdf = pd.read_parquet('../../time-use-survey-2024/clean/hh-tus-2024.parquet')
pdf = pd.read_parquet('../../time-use-survey-2024/clean/per-tus-2024.parquet')
pdf.dropna(subset=['time_from', 'time_to'], inplace=True)
pdf.reset_index(inplace=True)

In [2]:
start = pdf['time_from'].str.extract('(?P<hour>\d\d):(?P<minute>\d\d)').dropna().astype(int).apply(lambda x: datetime.now().replace(second=0, microsecond=0, **x), axis=1)
end = pdf['time_to'].str.extract('(?P<hour>\d\d):(?P<minute>\d\d)').dropna().astype(int).apply(lambda x: datetime.now().replace(second=0, microsecond=0, **x), axis=1)
pdf['time_spent'] = ((end - start).dt.total_seconds() / 3600).map(lambda x: x if x > 0 else 24 + x)

In [42]:
pdf['is_mixed'] = (pdf[['performed_multiple_activities', 'performed_simultaneous_activities']] == 'yes').any(axis=1)

In [6]:
PAID = [
    "self-employment for production of goods",  # Corresponds to code 13 [2, 4]
    "self-employment for production of services",  # Corresponds to code 14 [2, 4]
    "regular wage/salary for production of goods",  # Corresponds to code 15 [2, 4]
    "regular wage/salary for production of services",  # Corresponds to code 16 [2, 4]
    "casual labour for production of goods",  # Corresponds to code 17 [3, 4]
    "casual labour for production of services"  # Corresponds to code 18 [3, 4]
]
pdf['is_paid'] = pdf['paid_unpaid_status'].isin(PAID)
(pdf['is_paid'] * pdf['time_spent'] * pdf['mult']).sum() / pdf['mult'].sum()

0.1217673507072497

In [9]:
paid = pdf[pdf['is_paid']]
unpaid = pdf[~pdf['is_paid']]
(paid['time_spent'] * paid['mult']).sum() / paid['mult'].sum()

2.4984493638257614

In [10]:
(unpaid['time_spent'] * unpaid['mult']).sum() / unpaid['mult'].sum()

1.140787180771052

In [11]:
# That's how much time the average Indian spends on paid vs unpaid work

In [47]:
pdf.head()

schedule_id  schedule  \
fsu_serial_no sample_hh_no person_serial_no                         
32223         1            1                        TUS       106   
                           1                        TUS       106   
                           1                        TUS       106   
                           1                        TUS       106   
                           1                        TUS       106   

                                             survey_year sector  nss_region  \
fsu_serial_no sample_hh_no person_serial_no                                   
32223         1            1                        2024  Rural          11   
                           1                        2024  Rural          11   
                           1                        2024  Rural          11   
                           1                        2024  Rural          11   
                           1                        2024  Rural          11   

                                             district  stratum  sub_stratum  \
fsu_serial_no sample_hh_no person_serial_no                                   
32223         1            1                       21       13            1   
                           1                       21       13            1   
                           1                       21       13            1   
                           1                       21       13            1   
                           1                       21       13            1   

                                             sub_round  fod_sub_region  ...  \
fsu_serial_no sample_hh_no person_serial_no                             ...   
32223         1            1                         2             110  ...   
                           1                         2             110  ...   
                           1                         2             110  ...   
                           1                         2             110  ...   
                           1                         2             110  ...   

                                            major_minor_activity  \
fsu_serial_no sample_hh_no person_serial_no                        
32223         1            1                               major   
                           1                               major   
                           1                               major   
                           1                               major   
                           1                               major   

                                            activity_code  \
fsu_serial_no sample_hh_no person_serial_no                 
32223         1            1                        911.0   
                           1                        931.0   
                           1                        921.0   
                           1                        182.0   
                           1                        131.0   

                                                                      where_activity_performed  \
fsu_serial_no sample_hh_no person_serial_no                                                      
32223         1            1                 within premises of the dwelling unit of the se...   
                           1                 within premises of the dwelling unit of the se...   
                           1                 within premises of the dwelling unit of the se...   
                           1                 outside premises of the dwelling unit of the s...   
                           1                 outside premises of the dwelling unit of the s...   

                                                                            paid_unpaid_status  \
fsu_serial_no sample_hh_no person_serial_no                                                      
32223         1            1                 self development / self care / self maintenanc...   
            

In [49]:
pdf = pdf.reset_index().set_index(['fsu_serial_no', 'sample_hh_no', 'person_serial_no', 'activity_srl_no'], verify_integrity=True)

In [50]:
pdf.head()

schedule_id  \
fsu_serial_no sample_hh_no person_serial_no activity_srl_no               
32223         1            1                1.0                     TUS   
                                            2.0                     TUS   
                                            3.0                     TUS   
                                            4.0                     TUS   
                                            5.0                     TUS   

                                                             schedule  \
fsu_serial_no sample_hh_no person_serial_no activity_srl_no             
32223         1            1                1.0                   106   
                                            2.0                   106   
                                            3.0                   106   
                                            4.0                   106   
                                            5.0                   106   

                                                             survey_year  \
fsu_serial_no sample_hh_no person_serial_no activity_srl_no                
32223         1            1                1.0                     2024   
                                            2.0                     2024   
                                            3.0                     2024   
                                            4.0                     2024   
                                            5.0                     2024   

                                                            sector  \
fsu_serial_no sample_hh_no person_serial_no activity_srl_no          
32223         1            1                1.0              Rural   
                                            2.0              Rural   
                                            3.0              Rural   
                                            4.0              Rural   
                                            5.0              Rural   

                                                             nss_region  \
fsu_serial_no sample_hh_no person_serial_no activity_srl_no               
32223         1            1                1.0                      11   
                                            2.0                      11   
                                            3.0                      11   
                                            4.0                      11   
                                            5.0                      11   

                                                             district  \
fsu_serial_no sample_hh_no person_serial_no activity_srl_no             
32223         1            1                1.0                    21   
                                            2.0                    21   
                                            3.0                    21   
                                            4.0                    21   
                                            5.0                    21   

                                                             stratum  \
fsu_serial_no sample_hh_no person_serial_no activity_srl_no            
32223         1            1                1.0                   13   
                                            2.0                   13   
                                            3.0                   13   
                                            4.0                   13   
                                            5.0                   13   

                                                             sub_stratum  \
fsu_serial_no sample_hh_no person_serial_no activity_srl_no                
32223         1            1                1.0                        1   
                                            2.0                        1   
                                            3.0                        1   
                                            4.0                        1   

In [57]:
MAJOR_MINOR = pd.CategoricalDtype(['minor', 'major'], ordered=True)

pdf['major_minor_activity'] = pdf['major_minor_activity'].astype(MAJOR_MINOR)
xdf = pdf.sort_values(['time_from', 'major_minor_activity']).sort_index().drop_duplicates([pdf.index, 'time_from', 'time_to'], keep='last')

TypeError: unhashable type: 'MultiIndex'